In [1]:
import dspy
from dotenv import load_dotenv

In [12]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

In [13]:
# llm = dspy.OpenAI(model='gpt-3.5-turbo-0125', api_key=OPENAI_API_KEY, max_tokens=300)
llm = dspy.OllamaLocal("zephyr-7b-beta", model_type="text", stop=['\n\n'], max_tokens=300)
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts, temperature=0.7)

In [14]:
colbertv2_wiki17_abstracts

In [5]:
from dspy.datasets import HotPotQA


In [6]:
# Load dataset
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell Dspy that the 'question' field is the input. Any other fields are labels or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

/home/peter-legion-wsl2/peter-projects/daybreak-ml-docs/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [7]:
len(trainset), len(devset)

(20, 50)

In [10]:
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

In [8]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers"""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")
    

In [16]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [19]:
rag = RAG()

In [20]:
rag

generate_answer = ChainOfThought(GenerateAnswer(context, question -> answer
    instructions='Answer questions with short factoid answers'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [23]:
rag.retrieve("what is an apple")

AttributeError: 'RAG' object has no attribute 'retrieve'